<a href="https://colab.research.google.com/github/SandeshRangreji/Scientific-Document-Summarization/blob/main/BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install datasets
!pip install rouge_score
!pip install sacrebleu
!pip install bert_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import torch
import numpy as np
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer, T5ForConditionalGeneration, T5TokenizerFast
from transformers import TFTrainer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import xml.etree.ElementTree as ET
from google.colab import drive
import os
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
os.mkdir("/content/PGN")
os.mkdir("/content/PGN/data")

In [ ]:
!unzip "/content/drive/MyDrive/Pointer Generator Networks/Data/scisumm.zip" -d "/content/PGN/data/"

In [ ]:
path = "/content/PGN/data/scisummnet_release1.1__20190413/top1000_complete/"
dir_list = os.listdir(path)
print(len(dir_list))

1009


In [ ]:
def get_text(xmlfile):
 sections = []
 intro_flag = False
 conclusion_flag = False
 tree = ET.parse(xmlfile)
 root = tree.getroot()
 for child in root:
  text = ""
  if child.tag != "S":
   if (child.attrib.get("title", child.tag).find("Introduction") != -1) and not intro_flag:
    intro_flag = True
    for grandchild in child:
     text = text + " " + grandchild.text
   if (child.attrib.get("title", child.tag).find("Conclusion") != -1) and not conclusion_flag:
    conclusion_flag = True
    for grandchild in child:
     text = text + " " + grandchild.text
  if text != "":
   sections.append(text)
 return sections, (intro_flag and conclusion_flag)

In [ ]:
a, b = get_text("/content/PGN/data/scisummnet_release1.1__20190413/top1000_complete/A00-1031/Documents_xml/A00-1031.xml") 
print((len(a)))  

2


In [ ]:
def get_articles(dir_list):
  articles = []
  blacklist = []
  articles_index = []
  total = 0
  for file_name in dir_list:
    if file_name not in blacklist:
      xmlfile = "/content/PGN/data/scisummnet_release1.1__20190413/top1000_complete/" + file_name + "/Documents_xml/" + file_name + ".xml"
      try:
        sections, flag = get_text(xmlfile)
        if flag == False:
          total+=1
        else:
          articles.append(sections)
          articles_index.append(xmlfile)
      except:
        blacklist.append(file_name)
  return articles, total, articles_index

In [ ]:
def word_count(sent):
  return len(sent.split(" "))

In [ ]:
articles, total, articles_index = get_articles(dir_list)

In [ ]:
def get_reference_summaries(articles_index):
  reference_summaries = []
  for file_name in articles_index:
    start_index = file_name.find("top1000_complete/") + 17
    end_index = file_name.find("/Documents_xml")
    file_name = file_name[start_index:end_index]
    ref_sum_file = "/content/PGN/data/scisummnet_release1.1__20190413/top1000_complete/" + file_name + "/summary/" + file_name + ".gold.txt"
    reference_summary = open(ref_sum_file, "r")
    reference_summaries.append(reference_summary.read().replace("\n", " ").replace("  "," ") + " </s>")
  return reference_summaries

In [ ]:
reference_summaries = get_reference_summaries(articles_index)

In [ ]:
input_sequences = []
for article in articles:
  # input_sequences.append("summarize:" + " ".join(article) + " </s>")
  input_sequences.append(" ".join(article))

In [ ]:
average_length = 0
input_lengths = []
for inp in input_sequences:
  input_lengths.append(word_count(inp))
print(sum(input_lengths)/len(input_sequences))

685.726148409894


In [ ]:
output_lengths = []
for out in reference_summaries:
  output_lengths.append(word_count(out))
print(sum(output_lengths)/len(output_lengths))

150.4876325088339


In [ ]:
max(output_lengths)

314

In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
shuffle_dataset = {"input" : input_sequences, "labels" : reference_summaries}

In [ ]:
shuffle_dataset = Dataset.from_dict(shuffle_dataset)
shuffle_dataset.shuffle(seed=42)

Dataset({
    features: ['input', 'labels'],
    num_rows: 566
})

In [ ]:
# train_dataset_dict = {"input" : input_sequences[:450], "labels" : reference_summaries[:450]}
# eval_dataset_dict = {"input" : input_sequences[450:500], "labels" : reference_summaries[450:500]}
# test_dataset_dict = {"input" : input_sequences[500:], "labels" : reference_summaries[500:]}
train_dataset_dict = shuffle_dataset[:450]
eval_dataset_dict = shuffle_dataset[450:500]
test_dataset_dict = shuffle_dataset[500:]

In [ ]:
train_dataset = Dataset.from_dict(train_dataset_dict)
eval_dataset = Dataset.from_dict(eval_dataset_dict)
test_dataset = Dataset.from_dict(test_dataset_dict)

In [ ]:
train_dataset[0]

{'input': " Given a word, its context and its possible meanings, the problem of word sense disambiguation (WSD) is to determine the meaning of the word in that context. WSD is useful in many natural language tasks, such as choosing the correct word in machine translation and coreference resolution. In several recent proposals (Hearst, 1991; Bruce and Wiebe, 1994; Leacock, Towwell, and Voorhees, 1996; Ng and Lee, 1996; Yarowsky, 1992; Yarowsky, 1994), statistical and machine learning techniques were used to extract classifiers from hand-tagged corpus. Yarowsky (Yarowsky, 1995) proposed an unsupervised method that used heuristics to obtain seed classifications and expanded the results to the other parts of the corpus, thus avoided the need to hand-annotate any examples. Most previous corpus-based WSD algorithms determine the meanings of polysemous words by exploiting their local contexts. A basic intuition that underlies those algorithms is the following: (1) Two occurrences of the same 

In [ ]:
dataset = DatasetDict({
    'train' : train_dataset,
    'validation': eval_dataset,
    'test': test_dataset,
})

In [ ]:
max_input_length = 512 # 1536
max_target_length = 162 # 162
source = "input"
target = "labels"

In [ ]:
# from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
# tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
def preprocess_function(data):
  input = data[source]
  targets = data[target]
  model_inputs = tokenizer(input, max_length=max_input_length, truncation=True)
  labels = None
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
temp = preprocess_function(dataset['train'][:2])

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
batch_size = 1
source_lang = "article"
target_lang = "summary"
model_name = 'pegasus'
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=12,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    generation_max_length = 154
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")
BertScore = load_metric("bertscore")

Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def convert_rouge_to_dict(score):
  
  rouge1 = score["rouge1"]
  rouge2 = score["rouge2"]
  rougeL = score["rougeL"]
  return {
      "rouge1_precision": rouge1.mid.precision,
      "rouge1_recall": rouge1.mid.recall,
      "rouge1_fmeasure": rouge1.mid.fmeasure,
      "rouge2_precision": rouge2.mid.precision,
      "rouge2_recall": rouge2.mid.recall,
      "rouge2_fmeasure": rouge2.mid.fmeasure,
      "rougeL_precision": rougeL.mid.precision,
      "rougeL_recall": rougeL.mid.recall,
      "rougeL_fmeasure": rougeL.mid.fmeasure,
  }

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    print(result)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Using amp fp16 backend


In [ ]:
def generate_summaries(num_beams=10, num_return_sequences=10):
  torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  generated_summaries = []
  for article in test_dataset["input"]:
    batch = tokenizer([article],truncation=True,padding='longest',max_length=max_input_length, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch,max_length=max_target_length,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    generated_summaries.append(tgt_text[0])
  return generated_summaries

In [ ]:
def get_BertScore(BertScore, generated_summaries, test_dataset):
  predictions = generated_summaries
  references = test_dataset["labels"]
  results = BertScore.compute(predictions=predictions, references=references, lang="en")
  f1_score = [round(v, 2) for v in results["f1"]]
  precision = [round(v, 2) for v in results["precision"]]
  recall = [round(v, 2) for v in results["recall"]]
  return f1_score, precision, recall

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])
# recall: n-grams in the reference summary are present in the generated summary
# precision: n-grams in the generated summary are present in the reference summary

The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: input.
***** Running Evaluation *****
  Num examples = 66
  Batch size = 1


{'rouge1': AggregateScore(low=Score(precision=0.34941308120846976, recall=0.3294334985701574, fmeasure=0.33800718563886156), mid=Score(precision=0.3681033205636139, recall=0.3489958307330118, fmeasure=0.3562686795618909), high=Score(precision=0.3887017512490416, recall=0.36864634030958404, fmeasure=0.3751719552785516)), 'rouge2': AggregateScore(low=Score(precision=0.08758786749795573, recall=0.08212256090951187, fmeasure=0.08437877090782643), mid=Score(precision=0.10620085355194947, recall=0.10065973635412793, fmeasure=0.10256284704127348), high=Score(precision=0.12835576555402914, recall=0.12380812296793015, fmeasure=0.1251148800570058)), 'rougeL': AggregateScore(low=Score(precision=0.1860322090235625, recall=0.1757116298230002, fmeasure=0.17999398602491068), mid=Score(precision=0.2042417855876666, recall=0.19309326324683207, fmeasure=0.19728332606931487), high=Score(precision=0.2238011955339956, recall=0.2129063014744736, fmeasure=0.21642058567636668)), 'rougeLsum': AggregateScore(lo

{'eval_gen_len': 154.0,
 'eval_loss': 3.433584451675415,
 'eval_rouge1': 35.6269,
 'eval_rouge2': 10.2563,
 'eval_rougeL': 19.7283,
 'eval_rougeLsum': 31.8499,
 'eval_runtime': 77.2648,
 'eval_samples_per_second': 0.854,
 'eval_steps_per_second': 0.854}

In [ ]:
generated_summaries = generate_summaries()
f1_score, precision, recall = get_BertScore(BertScore, generated_summaries, test_dataset)
print("F1 score : ", sum(f1_score)/len(f1_score))
print("Precision : ", sum(precision)/len(precision))
print("Recall : ", sum(recall)/len(recall))

Could not locate the tokenizer configuration file, will try to use the model config instead.
https://huggingface.co/roberta-large/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp2akrvy5f


Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

storing https://huggingface.co/roberta-large/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
creating metadata file for /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
loading configuration file https://huggingface.co/roberta-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_r

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

storing https://huggingface.co/roberta-large/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/7c1ba2435b05451bc3b4da073c8dec9630b22024a65f6c41053caccf2880eb8f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
creating metadata file for /root/.cache/huggingface/transformers/7c1ba2435b05451bc3b4da073c8dec9630b22024a65f6c41053caccf2880eb8f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
https://huggingface.co/roberta-large/resolve/main/merges.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp_37vft6y


...

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

storing https://huggingface.co/roberta-large/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/e16a2590deb9e6d73711d6e05bf27d832fa8c1162d807222e043ca650a556964.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
creating metadata file for /root/.cache/huggingface/transformers/e16a2590deb9e6d73711d6e05bf27d832fa8c1162d807222e043ca650a556964.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
loading file https://huggingface.co/roberta-large/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/7c1ba2435b05451bc3b4da073c8dec9630b22024a65f6c41053caccf2880eb8f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-large/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/20b5a00a80e27ae9accbe25672aba42ad2d4d4cb2c4b9359b50ca8e34e107d6d.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/robe

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

storing https://huggingface.co/roberta-large/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/8e36ec2f5052bec1e79e139b84c2c3089cb647694ba0f4f634fec7b8258f7c89.c43841d8c5cd23c435408295164cda9525270aa42cd0cc9200911570c0342352
creating metadata file for /root/.cache/huggingface/transformers/8e36ec2f5052bec1e79e139b84c2c3089cb647694ba0f4f634fec7b8258f7c89.c43841d8c5cd23c435408295164cda9525270aa42cd0cc9200911570c0342352
loading weights file https://huggingface.co/roberta-large/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/8e36ec2f5052bec1e79e139b84c2c3089cb647694ba0f4f634fec7b8258f7c89.c43841d8c5cd23c435408295164cda9525270aa42cd0cc9200911570c0342352
All the weights of RobertaModel were initialized from the model checkpoint at roberta-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaModel for predictions without further training.


F1 score :  0.8365151515151515
Precision :  0.8359090909090913
Recall :  0.8369696969696969


In [ ]:
generated_summaries[2]

' Spelling errors are generally grouped into two classes (Kuckich, 1992) — typographic and cognitive. Cognitive errors occur when the writer does not know how to spell a word. In these cases the misspelling often has the same pronunciation as the correct word ( for example writing latex as latecks). Typographic errors are mostly errors related to the keyboard; e.g., substitution or transposition of two letters because their keys are close on the keyboard. Damerau (1964) found that 80% of misspelled words that are non-word errors are the result of a single insertion, deletion, substitution or Transposition of letters. Many of the early algorithms for spelling correction are based on the assumption that the correct words differs from the misspelled by exactly one of these operations (M'

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: input.
***** Running training *****
  Num examples = 450
  Num Epochs = 12
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 5400
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1357: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.008274,40.998200,13.205100,22.961500,36.691200,121.900000
2,3.374600,2.960455,42.159900,13.265900,23.019700,37.687400,139.340000
3,2.929800,2.979539,41.412500,12.422300,22.588500,36.980400,142.280000
4,2.688300,2.966652,43.091900,14.112400,23.874500,39.033300,132.640000
5,2.449900,2.978394,43.923800,13.740500,23.482100,39.099600,143.280000
6,2.335000,3.007717,42.076200,13.163000,22.888000,37.867500,142.380000
7,2.177300,3.019947,42.298700,12.788100,23.070900,37.165400,139.100000
8,2.072000,3.040539,43.259900,13.629400,23.391100,39.135400,148.740000
9,1.968800,3.072999,44.106300,14.216500,24.062100,39.471600,143.300000
10,1.906700,3.087852,42.774200,12.893100,23.012500,38.043000,147.280000


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: input.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 1


{'rouge1': AggregateScore(low=Score(precision=0.4489381394575528, recall=0.3472810106653962, fmeasure=0.3871896810445573), mid=Score(precision=0.48003277735110544, recall=0.37377860886636116, fmeasure=0.40998230748327824), high=Score(precision=0.5107402274226336, recall=0.39964834343964034, fmeasure=0.43144555721431455)), 'rouge2': AggregateScore(low=Score(precision=0.13615331105064374, recall=0.10403872098340498, fmeasure=0.11642930384411966), mid=Score(precision=0.1588148640831395, recall=0.11836641048414434, fmeasure=0.13205105834332664), high=Score(precision=0.1845794588853102, recall=0.13356301801139034, fmeasure=0.14887554855483354)), 'rougeL': AggregateScore(low=Score(precision=0.24736996775489206, recall=0.1932910224777271, fmeasure=0.21479023179220294), mid=Score(precision=0.2729019176149651, recall=0.2081824328822221, fmeasure=0.22961519593506163), high=Score(precision=0.30167480346052916, recall=0.22277297431198553, fmeasure=0.2445060453768898)), 'rougeLsum': AggregateScore(

Saving model checkpoint to pegasus-finetuned-article-to-summary/checkpoint-500
Configuration saved in pegasus-finetuned-article-to-summary/checkpoint-500/config.json
Model weights saved in pegasus-finetuned-article-to-summary/checkpoint-500/pytorch_model.bin
tokenizer config file saved in pegasus-finetuned-article-to-summary/checkpoint-500/tokenizer_config.json
Special tokens file saved in pegasus-finetuned-article-to-summary/checkpoint-500/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1357: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have

...

{'rouge1': AggregateScore(low=Score(precision=0.42198947844837126, recall=0.4151658554460215, fmeasure=0.4164016917608207), mid=Score(precision=0.4478323901152239, recall=0.43456202174425207, fmeasure=0.43517834895898533), high=Score(precision=0.4710079322490057, recall=0.4561113553051039, fmeasure=0.45445274108188927)), 'rouge2': AggregateScore(low=Score(precision=0.1221785821987855, recall=0.11805954123360496, fmeasure=0.11928597106039936), mid=Score(precision=0.1410426145699195, recall=0.13555898388158538, fmeasure=0.13638476459711654), high=Score(precision=0.16196969162851121, recall=0.1543891202009238, fmeasure=0.15529806701371154)), 'rougeL': AggregateScore(low=Score(precision=0.22387484306511637, recall=0.21848517821653363, fmeasure=0.2194853836445837), mid=Score(precision=0.24066424137613954, recall=0.23235126055847544, fmeasure=0.2330781506152272), high=Score(precision=0.26039140546639783, recall=0.24920656912660907, fmeasure=0.24884070215840579)), 'rougeLsum': AggregateScore(

Saving model checkpoint to pegasus-finetuned-article-to-summary/checkpoint-5000
Configuration saved in pegasus-finetuned-article-to-summary/checkpoint-5000/config.json
Model weights saved in pegasus-finetuned-article-to-summary/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in pegasus-finetuned-article-to-summary/checkpoint-5000/tokenizer_config.json
Special tokens file saved in pegasus-finetuned-article-to-summary/checkpoint-5000/special_tokens_map.json
Deleting older checkpoint [pegasus-finetuned-article-to-summary/checkpoint-3500] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: input.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 1


{'rouge1': AggregateScore(low=Score(precision=0.42267151286506904, recall=0.4242251199096263, fmeasure=0.4206430200337183), mid=Score(precision=0.4534789818244557, recall=0.44405773399246373, fmeasure=0.44193894222560476), high=Score(precision=0.4792306485104704, recall=0.4662739538520438, fmeasure=0.46186197876098456)), 'rouge2': AggregateScore(low=Score(precision=0.1246681642633789, recall=0.12205738190054569, fmeasure=0.1225974512060015), mid=Score(precision=0.14378542712455605, recall=0.13827935737937802, fmeasure=0.13890681692205453), high=Score(precision=0.16385812912450082, recall=0.15705679945601053, fmeasure=0.1572240706609954)), 'rougeL': AggregateScore(low=Score(precision=0.22133630939908483, recall=0.22162468683187372, fmeasure=0.21981924741436767), mid=Score(precision=0.23886990686460685, recall=0.23424958842863236, fmeasure=0.232470461522932), high=Score(precision=0.25620231049696013, recall=0.24860668773848948, fmeasure=0.24629074997676006)), 'rougeLsum': AggregateScore(



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=5400, training_loss=2.3344677734375, metrics={'train_runtime': 1378.0764, 'train_samples_per_second': 3.919, 'train_steps_per_second': 3.919, 'total_flos': 2103040589045760.0, 'train_loss': 2.3344677734375, 'epoch': 12.0})

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: input.
***** Running Evaluation *****
  Num examples = 66
  Batch size = 1


{'rouge1': AggregateScore(low=Score(precision=0.46187880474427834, recall=0.4332277681046539, fmeasure=0.4459110604215667), mid=Score(precision=0.4809026654805355, recall=0.4524367624492992, fmeasure=0.4628236528943966), high=Score(precision=0.500702437983396, recall=0.4733268156833602, fmeasure=0.4806273201299497)), 'rouge2': AggregateScore(low=Score(precision=0.15430869487213691, recall=0.14582052212073515, fmeasure=0.14906182867824952), mid=Score(precision=0.17669257326810395, recall=0.16751521094982258, fmeasure=0.17080690217214728), high=Score(precision=0.20046210279030266, recall=0.19289772840261118, fmeasure=0.19511606089914046)), 'rougeL': AggregateScore(low=Score(precision=0.25131392630980653, recall=0.2355234636574989, fmeasure=0.24258118532226472), mid=Score(precision=0.2692593034814067, recall=0.2534549719849259, fmeasure=0.2591041113180149), high=Score(precision=0.28902319025176004, recall=0.2751912520673958, fmeasure=0.2785848286289006)), 'rougeLsum': AggregateScore(low=S

{'epoch': 12.0,
 'eval_gen_len': 147.0152,
 'eval_loss': 2.9332096576690674,
 'eval_rouge1': 46.2824,
 'eval_rouge2': 17.0807,
 'eval_rougeL': 25.9104,
 'eval_rougeLsum': 42.0088,
 'eval_runtime': 77.9823,
 'eval_samples_per_second': 0.846,
 'eval_steps_per_second': 0.846}

In [ ]:
generated_summaries = generate_summaries()
f1_score, precision, recall = get_BertScore(BertScore, generated_summaries, test_dataset)
print("F1 score : ", sum(f1_score)/len(f1_score))
print("Precision : ", sum(precision)/len(precision))
print("Recall : ", sum(recall)/len(recall))

F1 score :  0.8607575757575757
Precision :  0.8651515151515154
Recall :  0.856818181818182


In [ ]:
# before training
# F1 score :  0.8512121212121213
# Precision :  0.8710606060606059
# Recall :  0.8319696969696969

# temp = 1.5
# F1 score :  0.8556060606060607
# Precision :  0.8637878787878792
# Recall :  0.8466666666666669

# temp = 2.0
# F1 score :  0.8556060606060607
# Precision :  0.8637878787878792
# Recall :  0.8466666666666669

In [ ]:
generated_summaries[2]

'A New Error Model For Phonetic Errors Spelling errors are generally grouped into two classes (Kuckich, 1992) — typographic and cognitive. In typographic errors occur when the writer does not know how to spell a word. In these cases the misspelling often has the same pronunciation as the correct word ( for example writing latex as latecks). This paper describes a new error model for phonetic errors, which is based on a letter-based model trained on the same data as the first model, but using the pronunciations of the correct words and (incorrect) guesses of the misspellings to learn phone-sequence-to-phone-sequence edits and estimate their error rate. This model reduces the error rate of the best previous error model by nearly 50%. We'

In [ ]:
test_dataset["labels"][2]

'Pronunciation Modeling For Improved Spelling Correction This paper presents a method for incorporating word pronunciation information in a noisy channel model for spelling correction. The proposed method builds an explicit error model for word pronunciations. By modeling pronunciation similarities between words we achieve a substantial performance improvement over the previous best performing models for spelling correction. We consider a pronunciation variation model to generate multiple pronunciations for each canonical pronunciation in a pronouncing dictionary. We extend Brill and Moore (2000) to consider edits over both letter sequences and sequences of phones in the pronunciations of the word and misspelling. We use the noisy channel model approach to determine the types and weights of edit operations. Since a spelling correction model needs to rank candidate words rather than candidate pronunciations, we derive an error model that determines the probability that a word w was spel